# Graph based response types

In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

client = AzureOpenAI(
        api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version = "2024-08-01-preview",
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

deployment_name = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")

In [ ]:
from pydantic import BaseModel, Field
from typing import List
import graphviz


class Node(BaseModel):
    id: int
    label: str
    color: str

class Edge(BaseModel):
    source: int
    target: int
    label: str
    color: str

class KnowledgeGraph(BaseModel):
    nodes: List[Node] = Field(default_factory=list)
    edges: List[Edge] = Field(default_factory=list)

    def visualize(self):
        dot = graphviz.Digraph(comment="Knowledge Graph")

        for node in self.nodes:
            dot.node(str(node.id), label=node.label, color=node.color)

        for edge in self.edges:
            dot.edge(str(edge.source), str(edge.target), label=edge.label, color=edge.color)
        
        dot.render("knowledge_graph.gv", view=True)
    

def generate_graph(input) -> KnowledgeGraph:
    completion = client.beta.chat.completions.parse(
        model = deployment_name,
        messages = [{"role" : "assistant", "content" : f""" Help me understand the following by describing as a detailed knowledge graph:  {input}"""}],
        response_format = KnowledgeGraph)
    
    print(completion)

    return completion.choices[0].message.parsed
    


In [ ]:
generate_graph("What is the relationship between cars, wheels and airplanes").visualize()